In [21]:
first_time = False

# Import Statements

In [22]:
if (first_time):
  from google.colab import drive
  drive.mount('/content/drive')

import collections
import pickle
import gensim
# from gensim.models.keyedvectors import KeyedVectors
import numpy as np
import re
import random
import sys
import os 
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors
from gensim.models.fasttext import FastText
from collections.abc import Iterable

# Data Paths

In [32]:
path_to_folder = '/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/'
embeddings_path = path_to_folder + 'word_embedding/fasttext/commen_docid_removed300_5'
keyed_vectors_path = path_to_folder + "word_embedding/fasttext/fasttext_keyed_vectors/puncuations_removed/common_docid_removed_keyed.kv"
dataset_path = path_to_folder + "corpus/analyzed/lankadeepa_tagged_2.csv"

# Implementation

In [24]:
def insert_word(dataset, all_words):
    for lines in dataset:
      if isinstance(lines, Iterable):
        for l in lines:
          if isinstance(l, Iterable):
            all_words+=l

In [25]:
def convert_words_to_number(dataset_text, dataset_label, common_word):
	transformed_text=[]
	transformed_label=[]
	for lines, labels in zip(dataset_text, dataset_label):
		new_x=[]
		new_label=[]
		for l, label in zip(lines,labels):
			words=[common_word[w] if w in common_word else 1 for w in l]	
			
			new_x+=[words]
			new_label+=[label]

		transformed_text+=[new_x]
		transformed_label+=[new_label]
	return transformed_text, transformed_label

In [26]:
def text_preprocessing(train_data,test_data):
  train_data_texts = train_data['comment']
  train_data_labels = train_data['label']
  test_data_texts = test_data['comment']
  test_data_labels = test_data['label']


  comment_texts = []
  comment_labels = []

  train_text = []
  test_text = []
  train_labels=[]
  test_labels=[]

  for label in train_data_labels:
    if label == "POSITIVE":
      train_labels.append(1)
    else:
      train_labels.append(0)
  comment_labels.append(train_labels)

  for label in test_data_labels:
    if label == "POSITIVE":
      test_labels.append(1)
    else:
      test_labels.append(0)
  comment_labels.append(test_labels)
  

  for comment in train_data_texts:
    lines = []
    try:
      words = comment.split()
      lines += words
    except:
      continue
    train_text.append(lines)
    
  comment_texts.append(train_text)

  for comment in test_data_texts:
    lines = [] 
    try:
      words = comment.split()
      lines += words
    except:
      continue
    test_text.append(lines)
    
  comment_texts.append(test_text)


  return comment_texts,comment_labels

In [27]:
def generate_keyed_vectors(embeddings_path,keyed_vectors_path):
  embedding_model = FastText.load(embeddings_path)
  word_vectors = embedding_model.wv
  word_vectors.save(keyed_vectors_path)
  return word_vectors

In [28]:
def check_model_qulity(model, word):
    for s in model.most_similar(word):
        print(s[0])

# Main

## Load Word Embeddings

In [ ]:
# word_vectors = generate_keyed_vectors(embeddings_path)
word_vectors = KeyedVectors.load(keyed_vectors_path, mmap='r')
word_vectors['නැහැ']

In [30]:
check_model_qulity(word_vectors,'අධිරාජ්‍යයා')

අධිරාජ්‍යය
අධිරාජ්‍යයාගේ
අධිරාජ්‍යා
අධිරාජ්‍යයටම
අධිරාජ්‍යාට
අධිරාජ්‍යයේ
අධිරාජ්‍යයවාදි
අධිරාජ්‍යයට
අධිරාජ්‍ය
අධිරාජ්‍යට


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [33]:
#allData = pd.read_csv("/content/drive/My Drive/University/FYP/Sentiment Analysis/Sentiment-tagger/corpus/analyzed/comments_all.csv", ";")
DataTagged = pd.read_csv(dataset_path, ";")

print(DataTagged.columns.values)

train_data, test_data = train_test_split(DataTagged, test_size=0.2, random_state=0)


['docid' 'comment' 'label']


In [41]:
test_data

,docid,comment,label
4380,44501,මේවා සුරංගනා කතා,POSITIVE
2058,249738,මේ තරුණයා ලැබුන තෑගී මුදල් ආදිය යහපත් ආයෝජනයක ...,POSITIVE
2921,328955,බොහොම උසස් දෙයක් තම පියා සිහිවීම පිණිස තමා ඉගෙ...,POSITIVE
3239,361116,ඔබතුමාට නිවන් සුව අත් වේවා,POSITIVE
1172,203000,ලැජ්ජයි,NEGATIVE
...,...,...,...
1098,198886,ලංකාදීප එසැන පුවත් මිතුරන්ටත් සියලු සුභ පැතුම්,POSITIVE
2510,273890,මෙවන් මොහොතක ඔබ වැනි විරුවෙකුගේ පැමිණීම ඔවුන්ට...,POSITIVE
2924,328955,මටත් ඒකමයි හිතුනේ,POSITIVE
401,125388,තෙරුවන් සරන ලැබේවා ඔබ සැමට,POSITIVE


In [34]:
comment_texts, comment_labels = text_preprocessing(train_data,test_data)

In [35]:
#insert all words
all_words=[]
insert_word(comment_texts, all_words)

In [36]:
#obtain frequent words
counter=collections.Counter(all_words)
vocab=len(counter)
vocab_size=vocab-2	
common_word=dict(counter.most_common(vocab_size))
print(len(common_word))

18413


In [37]:
print(dict(counter.most_common(vocab_size)))

{'මේ': 1098, 'අපේ': 566, 'නම්': 554, 'නැහැ': 514, 'වගේ': 444, 'ඒ': 413, 'ඔබ': 368, 'අපි': 365, 'ගැන': 337, 'එක': 328, 'කරන්න': 302, 'රටේ': 284, 'හරි': 282, 'තමයි': 271, 'මම': 269, 'අය': 264, 'කියලා': 252, 'කරන': 249, 'ඇති': 242, 'දැන්': 238, 'ඔය': 235, 'හොඳ': 229, 'ලංකාවේ': 225, 'ඉන්න': 223, 'මිනිස්සු': 221, 'රට': 217, 'නැති': 207, 'නිසා': 199, 'වෙන්න': 196, 'අනේ': 192, 'වැඩ': 191, 'වැඩක්': 187, 'ඔබට': 183, 'මේක': 182, 'වේවා': 177, 'දෙයක්': 174, 'තියෙන්නේ': 174, 'කියන්නේ': 172, 'ගන්න': 172, 'වලට': 166, 'මගේ': 165, 'එපා': 163, 'අද': 160, 'කරලා': 158, 'අපිට': 156, 'අයට': 155, 'මට': 154, 'රටට': 152, 'දේවල්': 152, 'කියල': 150, 'ලැබේවා': 150, 'සහ': 147, 'මේවා': 147, 'පුළුවන්': 147, 'වෙන': 144, 'වෙනවා': 143, 'කරනවා': 141, 'වෙලා': 140, 'හොද': 139, '?': 139, 'සුබ': 138, 'යන්න': 134, 'හරිම': 134, 'ගොඩක්': 131, 'යන': 129, 'දෙන්න': 128, 'කළ': 127, 'නෑ': 122, 'කියන': 122, 'වල': 122, 'බව': 118, 'වඩා': 117, 'තව': 114, 'වෙන්නේ': 112, 'කරන්නේ': 112, 'ලොකු': 108, 'සල්ලි': 108, 'ශ්\u200dරී': 107, 'කෙනෙක

In [38]:
#number them
c=2
for key in common_word:
  common_word[key]=c
  c+=1

In [39]:
common_word['අපේ']

3

In [ ]:
transformed_text, transformed_label= convert_words_to_number(comment_texts, comment_labels, common_word)

In [ ]:
glove_dict={}
for word, vocab_obj in word_vectors.vocab.items():
  glove_dict[word]=word_vectors[word]

glove_dict['මම']

In [ ]:
i = 0
for number, word in sorted(zip(common_word.values(), common_word.keys())):
  print(number,word)
  i+=1
  if (i>5):
    break

2 මේ
3 අපේ
4 නම්
5 නැහැ
6 වගේ
7 ඒ


In [ ]:
# from preprocessing

transformed_text, transformed_label= convert_words_to_number(comment_texts, comment_labels, common_word)

pickle.dump(((transformed_text, transformed_label)), open('/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/SLSTM/parsed_data/from_fasttext/data_set_commn_docid_removed', 'wb'))
glove_dict={}
for word, vocab_obj in word_vectors.vocab.items():
  glove_dict[word]=word_vectors[word]

word2vec=[np.random.normal(0, 0.1, 300).tolist(), np.random.normal(0, 0.1, 300).tolist()]
missing=0
for number, word in sorted(zip(common_word.values(), common_word.keys())):
  try:
    word2vec.append(glove_dict[word])
  except KeyError: 	
    word2vec.append(np.random.normal(0, 0.1, 300).tolist())
    missing+=1
	#print(number)
  # print(len(word2vec))
pickle.dump(word2vec, open('/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/SLSTM/parsed_data/from_fasttext/fasttext_vectors_common_docid_removed', 'wb'))
print(missing)
print(np.array(word2vec).shape)

2210
(18415, 300)


In [ ]:
cd /content/drive/My Drive/University/FYP/Sentiment Analysis/Sentiment-tagger/corpus/analyzed/dimuthu

/content/drive/My Drive/University/FYP/Sentiment Analysis/Sentiment-tagger/corpus/analyzed/dimuthu


In [ ]:
!unzip data.zip

Archive:  data.zip
  inflating: test_1_no_stop.txt      
  inflating: test_2.txt              
  inflating: test_2_no_stop.txt      
  inflating: test_3.txt              
  inflating: test_3_no_stop.txt      
  inflating: test_4.txt              
  inflating: test_4_no_stop.txt      
  inflating: test_5.txt              
  inflating: test_5_no_stop.txt      
  inflating: test_6.txt              
  inflating: test_6_no_stop.txt      
  inflating: test_7.txt              
  inflating: wikipedia_si_filtered_1.txt  
